<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/03_local_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Local Search Algorithms

Here, we want to find the shortest path connecting several cities considering the aerial distances. This problem is known as the "Traveling Sales(man) Problem".

If we have 15 cities to connect, we have 15! different possibilities. This is already bigger than 10^12. You can easily see that the problem becomes complex very quickly. We will have problems to systematically explore the search space. Therefore, we will use local search algorithms to tackle this problem.

Local search algorithms start with a solution and try to improve the solution by considering the neighbouring states. The best neighbour will be chosen until no better can be found.


Implement your local search algorithm of choice (your version of the simulated annealing, hill-climing or genetic algorithm) to find the shortest path connecting a list of cities!

For the **Testat**, you need to find the shortest path between the following cities:

`path = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']`


For each algorithm, I wrote down some hints and implementation suggestions below. You don't need to implement all algorithms to solve the testat exercise. Try to tweak your algorithm such that a good (or even best) solution is found.


In [ ]:
!git clone https://github.com/iaherzog/search.git

Cloning into 'search'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), 594.11 KiB | 2.28 MiB/s, done.


In [ ]:
import sys
sys.path.append('/content/search')

## General hints

 You can use the following helper functions to plot visualize your path and to evaluate its cost.

In [ ]:
import folium
map_ch = folium.Map(location=[46.8, 8.33],
                    zoom_start=8, tiles="Stamen TonerBackground")


def create_map(path, sbb, map):
    points = []
    first_city = path[0]
    for city in path:
        points.append([sbb.hubs[city].x, sbb.hubs[city].y])
        folium.Marker([sbb.hubs[city].x, sbb.hubs[city].y], popup=city).add_to(map)
    points.append([sbb.hubs[first_city].x, sbb.hubs[first_city].y])
    #folium.PolyLine(points, color='red').add_to(map)
    return map

def evaluate_path(path, distance_function):
    length = 0
    last_city = ""
    for city in path:
        if last_city == "":
            first_city = city;
        if last_city != "":
            length += distance_function(last_city, city)
        last_city = city;
    length += distance_function(first_city, last_city)
    return length


Let's import the data from sbb and viualize our initial path.

In [ ]:
from sbb import SBB

sbb = SBB()
sbb.import_data('/content/search/linie-mit-betriebspunkten.json')
distance_function = sbb.get_distance_between

path = ['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']

print("path cost = " + str(evaluate_path(path, distance_function)))

print(path)
m = create_map(path, sbb, map_ch)
m

successfully imported 2787 hubs
successfully imported 401 train lines
path cost = 2009.1265150575216
['Sursee', 'Sion', 'Altdorf', 'Landquart', 'Konolfingen', 'Thun', 'Twann', 'Sargans', 'Lausanne', 'Vevey', 'Locarno', 'Hinwil', 'Bern', 'Liestal', 'Lugano']


This is defenitly not the best way how to connect the cities. Find the optimal solution with **one** of the follwoing algorithms.

## Hill Climbing

Here, we try to minimize the distance of our path. So instead of hill climbing, we will do the opposite. Instead of trying to find the highest hill (maximum), we're looking at the deepest valley (minimum). But that's not a concern, we can easily change the sign to switch from a maximization  to a minimization problem.

*Hints:*
- use the `evaluate_path()` function we have defined earlier
- make sure to copy lists or sets properly: `current_path = path.copy()`
- you can convert sets to lists by `list(my_set)`
- a neighbouring path can be found by switching the position of two cities

In [ ]:
def hill_climbing_TSP(path, distance_function):
    return None


In [ ]:
best_path = hill_climbing_TSP(path, distance_function)
print("with length " + str(evaluate_path(best_path,sbb)))

TypeError: ignored

Oh, what happend here? Is this the best we can get?
- Why is this so?
- How many steps did we need to get to this solution?
- Try to improve the hill climbing algorithm with one of the methods you have seen in class?

## Genetic Algorithm

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

A genetic algorithm works in the following way:

1) Initialize random population.

2) Calculate population fitness.

3) Select individuals for mating.

4) Mate selected individuals to produce new population.

     * Random chance to mutate individuals.

5) Repeat from step 2) until an individual is fit enough or the maximum number of iterations was reached.

Below, you can find some helper functions to implement your genetic algorithm.

First, create a dictionnary that maps a letter to a city name.

Our solution will be a path through all the cities. To simplify, we will encode each city with a letter from the alphabet. So your first initial path through the cities will have the code "ABCDEFGHIJK..". We can easily convert a letter to a city by `letter2city('A')` or `city2letter('Rotkreuz')`.

In [ ]:
import string

number_of_cities = len(path)
letter2city = dict()
city2letter = dict()

for i in range(number_of_cities):
    letter2city[string.ascii_uppercase[i]] = list(path)[i]
    city2letter[list(path)[i]] = string.ascii_uppercase[i]

def path2string(path):
    s = ""
    for city in path:
        s+=city2letter[city]
    return list(s)

def path2cities(path):
    s = list()
    for letter in path:
        s.append(letter2city[letter])
    return s

path_code = path2string(path)
print("the path has the following code : ")
print(path_code)

the path has the following code : 
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']


Let's inizialize a random population:

In [ ]:
import random

def init_population(pop_number, cities):
    """Initializes population for genetic algorithm
    pop_number  :  Number of individuals in population
    cities      :  cities in letter code """



We can calculate the fitness of a path using the `evaluate_path` function. Note that shorter paths are considered fitter.

In [ ]:
def fitness(sample):
    return

Create a function to select two individuals for mating. Fitter individuals are more likely to be selected for reproduction than less fit individuals. Therefore, we have to calculate the weights of each indiviudal that corresponds to the likelyhood of being chosen for reproduction.

Now that we can select two individuals, we make them reproduce using crossover and mutation. We need to consider that we want to visit every city exactly once. For example, for the crossover, you can take a random lenght of individual 1 and fill up the remaining cities based on the order of the unvisited cities in individual 2.

In [ ]:
def crossover(x, y):
    # create an offspring from the parents x and y
    return

def mutate(x, p_mutate):
    # switch the location of two cities
    return

# test your code
x = path_code
y = random.sample(path_code, len(path_code))
xy = crossover(x,y)
print(x)
print(y)
print(xy)
mutate(xy, 0.5)
print(xy)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
['A', 'N', 'D', 'F', 'B', 'M', 'I', 'E', 'K', 'O', 'G', 'H', 'J', 'C', 'L']
None
None


We have now all the ingredients to create our genetic algorithm:

## Simulated Annealing


Simulated Annealing is quite similar to Hill Climbing,
but instead of picking the _best_ move every iteration, it picks a _random_ move.
If this random move brings us closer to the global optimum, it will be accepted,
but if it doesn't, the algorithm may accept or reject the move based on a probability dictated by the _temperature_.
When the *temperature* is high, the algorithm is more likely to accept a random move even if it is bad.
At low temperatures, only good moves are accepted, with the occasional exception.
This allows exploration of the state space and prevents the algorithm from getting stuck at a local optimum.

The temperature is gradually decreased over the course of the iteration.
This is done by a scheduling routine:


In [ ]:
import math
def exp_schedule(t, k=300, lam=0.0001, limit=20000):
    """One possible schedule function for simulated annealing"""
    return (k * math.exp(-lam * t) if t < limit else 0)

With this, try to implement the simulated annealing algorithm:

In [ ]:
def simulated_annealing_TSP(path, distance_function):
    return